In [2]:
%matplotlib widget
from util.sys import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi, meijering
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from skimage.feature import hessian_matrix_det
from experiment_class_surf import Experiment,clean_exp_with_hyphaes, Hyphae
from hyphae_id_surf import clean_and_relabel, get_mother, save_hyphaes, resolve_ambiguity_two_ends,solve_degree4, clean_obvious_fake_tips
from realign import transform_skeleton_final


In [3]:
plate = 3
begin = 0
end = 19
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[begin:end+1]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
exp = pickle.load( open(f'/scratch/shared/mrozemul/Fiji.app/Analysis_Plate{plate}_{dates[0]}_{dates[-1]}/experiment_{plate}.pick', "rb" ) )

In [48]:
def get_hyphae(experiment,exclude_bottom_factor=0.98):
    tips = [node for node in experiment.nodes if node.degree(node.ts()[0])==1 and node.pos(node.ts()[0])[0]<=experiment.boundaries_x[1]*exclude_bottom_factor]
    problems=[]
    small_problems = []
    hyphaes=[]
#     for i in range(20):
    for i,tip in enumerate(tips):
        if i%200==0:
            print(i/len(tips))
#         tip = choice(tips)
        hyphae = Hyphae(tip)
        roots = []
        for t in tip.ts():
#             print(t,tip)
            if tip.degree(t)==1:
                root,edges,nodes = hyphae.get_edges(t,100)
                roots.append(root)
        occurence_count = Counter(roots)
        if len(occurence_count.values())>=2:
            small_problems.append(tip)
        if len(occurence_count.values())>=2 and occurence_count.most_common(2)[0][0]!=roots[0] and occurence_count.most_common(2)[1][1]/occurence_count.most_common(2)[0][1]>=0.75:
            problems.append(tip)
        else:
            hyphae.root = occurence_count.most_common(2)[0][0]
            hyphae.ts = sorted(set(hyphae.ts).intersection(set(hyphae.root.ts())))
            hyphaes.append(hyphae)
    print(f'Detected problems during hyphae detection, {len(small_problems)} hyphaes have inconsistent root over time')
    print(f'Detected problems during hyphae detection, {len(problems)} hyphaes have inconsistent root over time')

    return(hyphaes,problems,small_problems)

In [49]:
hyphaes,problems,small_problems = get_hyphae(exp)

0.0
0.03496503496503497
0.06993006993006994
0.1048951048951049
0.13986013986013987
0.17482517482517482
0.2097902097902098
0.24475524475524477
0.27972027972027974
0.3146853146853147
0.34965034965034963
0.38461538461538464
0.4195804195804196
0.45454545454545453
0.48951048951048953
0.5244755244755245
0.5594405594405595
0.5944055944055944
0.6293706293706294
0.6643356643356644
0.6993006993006993
0.7342657342657343
0.7692307692307693
0.8041958041958042
0.8391608391608392
0.8741258741258742
0.9090909090909091
0.9440559440559441
0.9790209790209791
Detected problems during hyphae detection, 2648 hyphaes have inconsistent root over time
Detected problems during hyphae detection, 102 hyphaes have inconsistent root over time


In [47]:
problems

[]

In [39]:
experiment = exp
tips = [node for node in experiment.nodes if node.degree(node.ts()[0])==1 and node.pos(node.ts()[0])[0]<=experiment.boundaries_x[1]*0.98]
small_problems = []
hyphaes=[]
tip =choice(tips)
hyphae = Hyphae(tip)
roots = []
tss=[]
for t in tip.ts():
#             print(t,tip)
    if tip.degree(t)==1:
        tss.append(t)
        root,edges,nodes = hyphae.get_edges(t,100)
        roots.append(root)

In [44]:
roots[10:15],tss

([Node(3604), Node(3604), Node(201), Node(3604), Node(3604)],
 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

In [43]:
plt.close('all')
begin=10
end=15
exp.plot(tss[begin:end],[[node.label,tip.label] for node in roots][begin:end])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
tip.show_source_image(tip.ts()[10],tip.ts()[12])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …